In [1]:
from env.arm import TwoLinkArmEnv

import gym
import time
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt


from misc.random_agent import *
env = gym.make('CartPoleCont-v0')
sim_env = gym.make('CartPoleCont-v0')

## Random Agent

In [35]:
env.reset()
random_agent = RandomAgent(env.action_space)
for _ in range(100):
    env.render(mode='human')
    env.step(random_agent.act()) # take a random action
env.close()

/home/mjhwang/anaconda3/envs/tf2/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [36]:
envtr = gym.make('CartPoleCont-v0')
envtr._max_episode_steps = 500
env = gym.wrappers.Monitor(envtr, "~/Desktop/gym-agents/videos", force=True)
env.reset()
random_agent = RandomAgent(env.action_space)
for _ in range(50):
    env.render(mode='human')
    time.sleep(0.01)
    observation, reward, done, info = env.step(random_agent.act()) # take a random action
    if done:
        print("Episode finished after {} timesteps".format(_+1))
        break
        
env.stats_recorder.save_complete()
env.stats_recorder.done = True
env.close()
envtr.close()

Episode finished after 18 timesteps


## LQR Agent

In [6]:
env.reset()
goal = np.array([0, 0, 0, 0])
for _ in range(50):
    env.render()
    time.sleep(0.05)
    
    u = calc_lqr_input(env,sim_env,goal)
    state,re,done,_ = env.step(u)
env.close()

In [34]:
envtr = gym.make('CartPoleCont-v0')
envtr._max_episode_steps = 500
env = gym.wrappers.Monitor(envtr, "~/Desktop/gym-agents/videos", force=True)
env.reset()
for _ in range(50):
    env.render(mode='human')
    time.sleep(0.01)
    u = calc_lqr_input(env,sim_env,goal)
    state,re,done,_ = env.step(u)
    if done:
        print("Episode finished after {} timesteps".format(_+1))
        break
        
env.stats_recorder.save_complete()
env.stats_recorder.done = True
env.close()
envtr.close()

In [4]:
from scipy.linalg import solve_continuous_are as sol_riccatti

def simulate_dynamics(sim_env, x, u):
    sim_env.state = x.copy()
    x_1 = np.array(sim_env.step(u)[0])
    return x_1-x.copy()


def approximate_A(sim_env, x, u, delta=1e-5):
    A = np.zeros((x.shape[0], x.shape[0]))
    for i in range(A.shape[1]):

        x_perturbed = x.copy()
        x_perturbed[i] -= delta

        pert_neg = simulate_dynamics(sim_env,x_perturbed,u)

        x_perturbed = x.copy()
        x_perturbed[i] += delta
        pert_pos = simulate_dynamics(sim_env,x_perturbed,u)
        
        A[:,i] = (pert_pos - pert_neg)/(2*delta)

    return A


def approximate_B(sim_env, x, u, delta=1e-5):
    B = np.zeros((x.shape[0], u.shape[0]))
    for i in range(B.shape[1]):

        u_perturbed = u.copy()
        u_perturbed[i] -= delta
        pert_neg = simulate_dynamics(sim_env,x,u_perturbed)

        u_perturbed = u.copy()
        u_perturbed[i] += delta
        pert_pos = simulate_dynamics(sim_env,x,u_perturbed)

        B[:,i] = (pert_pos - pert_neg)/(2*delta)

    return B

def calc_lqr_input(env, sim_env, goal, u=None):
    if u is None:
        u = np.zeros(env.action_space.shape)

    x = env.state
    A = approximate_A(sim_env,x,u)
    B = approximate_B(sim_env,x,u)
    Q = np.eye(x.shape[0])
    R = np.eye(env.action_space.shape[0])
    P = sol_riccatti(A, B, Q, R)
    K = np.linalg.inv(R) @ B.T @ P
    u = -K @ (x-np.copy(goal))
    return u


In [6]:
env.rest()

(array([[6.053842 ],
        [2.9766529],
        [5.461025 ],
        [5.1331835]], dtype=float32),
 0.0,
 True,
 {})

In [23]:
env = gym.make('CartPole-v0')
env.reset()
env.state

array([ 0.03948536, -0.03347131,  0.02668339, -0.00560818])

## iLQR Agent

In [27]:
simulate_dynamics(env, env.state, 1)

[ 0.00758169  0.21961814  0.04678902 -0.27272491]
[ 0.03881593  0.161258    0.02657122 -0.28975419]


array([-0.03123424,  0.05836014,  0.0202178 ,  0.01702927])

In [31]:
approximate_A(env, env.state, 1)

array([[ 1041.12155488, -2082.77649132,  1086.50738601,  -872.89543614],
       [-1220.9859048 ,   866.28643259,  -622.98456538,   427.75968439],
       [ 2430.47469507, -2309.27268875,  3346.34181872,   435.24687625],
       [ 2008.52730615,  -482.10583511,  -356.61508217,  1930.8843898 ]])

In [37]:
approximate_A(env, env.state, 0)

array([[ -607.90344279,  -995.11432872,   189.97784244,    98.27222031],
       [-2002.56935536,  1342.99156751,   -62.52515566,  -901.60484931],
       [ -301.73213514,  1994.82534941,  1127.01438306,  -866.47303639],
       [ 1038.56983833,   -79.83316563,   212.52812192,  2408.66801234]])

In [30]:
def simulate_dynamics(env, x, u):
    env.reset()
    env.state = np.array(x).copy()
    x_1 = env.step(u)[0]
    return x_1 - x

def approximate_A(sim_env, x, u, delta=1e-5):
    A = np.zeros((x.shape[0], x.shape[0]))
    for i in range(A.shape[1]):
        x_perturbed = np.array(x).copy()
        x_perturbed[i] -= delta
        # print('x_neg', x_perturbed)
        pert_neg = simulate_dynamics(sim_env,x_perturbed,u)
        # print('out_neg',pert_neg)
        x_perturbed = np.array(x).copy()
        x_perturbed[i] += delta
        # print('x_pos', x_perturbed)
        pert_pos = simulate_dynamics(sim_env,x_perturbed,u)
        # print('out_neg',pert_pos)
        A[:,i] = (pert_pos - pert_neg)/(2*delta)
    return A

In [ ]:
reset

In [38]:
approximate_A(env, env.state, env.action_space.sample())

ValueError: operands could not be broadcast together with shapes (3,) (2,) 

In [32]:
def simulate_dynamics_next(env, x, u):
    env.state = x.copy()
    x_1 = env.step(u)[0]
    return x_1

In [33]:
def approx_F(env, x, u, delta=1e-5, n=10):
    v = np.concatenate((x, u))
    X = np.zeros((n, v.shape[0]))
    Y = np.zeros((n, x.shape[0]))
    x_1 = simulate_dynamics_next(env, x, u)
    for i in xrange(n):
        pert = np.random.normal(size=v.shape)
        pert /= np.sqrt(np.sum(pert * pert))
        pert *= delta
        v_p = v + pert
        X[i] = pert
        Y[i] = simulate_dynamics_next(env, v_p[:x.shape[0]], v_p[x.shape[0]:]) - x_1
    return scipy.linalg.lstsq(X, Y)[0]

In [16]:
simulate_dynamics_next(env, env.state, env.action_space.sample())

array([-0.67947638, -0.73369739,  0.75307987])

In [22]:
simulate_dynamics_next(env, env.state, env.action_space.sample())

array([-0.89310978, -0.44983878, -2.43259836])

In [23]:
def approx_F(env, x, u, delta=1e-5, n=10):
    v = np.concatenate((x, u))
    X = np.zeros((n, v.shape[0]))
    Y = np.zeros((n, x.shape[0]))
    x_1 = simulate_dynamics_next(env, x, u)
    for i in xrange(n):
        pert = np.random.normal(size=v.shape)
        pert /= np.sqrt(np.sum(pert * pert))
        pert *= delta
        v_p = v + pert
        X[i] = pert
        Y[i] = simulate_dynamics_next(env, v_p[:x.shape[0]], v_p[x.shape[0]:]) - x_1
    return scipy.linalg.lstsq(X, Y)[0]


In [4]:
env.step(env.action_space.sample())

(array([ 0.82319097,  0.56776459, -3.58017371]), -2.074779441779047, False, {})

In [8]:
def simulate_dynamics_next(env, x, u):
    env.state = x.copy()
    next_x,_,_,_ = env.step(u)
    return next_x.copy()


def cost_inter(env, x, u,lamb = 1.0):
    l = np.dot(u,u)*lamb
    l_x = np.zeros_like(x)*lamb
    l_xx = np.zeros([x.shape[0],x.shape[0]])*lamb
    l_u = 2*u*lamb
    l_uu = 2*np.eye(u.shape[0])*lamb
    l_ux = np.zeros([u.shape[0],x.shape[0]])*lamb
    return l,l_x.copy(),l_xx.copy(),l_u.copy(),l_uu.copy(),l_ux.copy()


def cost_final(env, x,lamb = 1.0):
    l = 10000*np.square(np.linalg.norm(x-env.goal,2))*lamb
    l_x = 20000*(x-env.goal)*lamb
    l_xx = 20000*np.eye(x.shape[0])*lamb
    return l,l_x.copy(),l_xx.copy()


def approximate_F(env,x,u,delta = 1e-5):

    #env.state = x.copy()
    con = np.hstack([x,u])
    x_next = simulate_dynamics_next(env,x,u)
    F = np.zeros([x.shape[0],con.shape[0]])
    for j in range(con.shape[0]):
      con_pert = con.copy()
      con_pert[j] += delta
      x_next_pert = simulate_dynamics_next(env,con_pert[:x.shape[0]],con_pert[x.shape[0]:])
      x_next_delta = x_next_pert-x_next
      for i in range(x.shape[0]):
        F[i,j] = x_next_delta[i]/delta
    return F.copy()

def simulate(env, x0, U):
    return None


def calc_ilqr_input(env, sim_env, tN=100, max_iter=1e5):
    """Calculate the optimal control input for the given state.
    Parameters
    ----------
    env: gym.core.Env
      This is the true environment you will execute the computed
      commands on. Use this environment to get the Q and R values as
      well as the state.
    sim_env: gym.core.Env
      A copy of the env class. Use this to simulate the dynamics when
      doing finite differences.
    tN: number of control steps you are going to execute
    max_itr: max iterations for optmization
    Returns
    -------
    U: np.array
      The SEQUENCE of commands to execute. The size should be (tN, #parameters)
    """
    
    x_0 = env.state.copy()
    adim = env.action_space.shape[0]
    xdim = x_0.shape[0]
    x_hat = np.zeros([tN+1,xdim])
    u_hat = np.zeros([tN,adim])
    # if init_u is not None:
    #   u_hat = init_u.copy()
    #   sim_env.state = x_0.copy()
    #   for i in range(tN):
    #     x_hat[i] = sim_env.state.copy()
    #     sim_env.step(u_hat[i])
    
    
    K_t = np.zeros([tN,adim,xdim])
    k_t = np.zeros([tN,adim])
    # Q_t = np.zeros([tN,x_dim+adim,x_dim+adim])
    # q_t = np.zeros([tN,x_dim+adim])
    V_t = np.zeros([tN+1,xdim,xdim])
    v_t = np.zeros([tN+1,xdim])
    # v = np.zeros([tN+1,])
    # v_x = np.zeros([tN+1,xdim])
    # v_xx = np.zeros([tN+1,xdim,xdim])
    
    total_cost = []
    total_reward = []

    for i in range(int(max_iter)):
      print("iteration: %d"%(i+1))
      rtotal = 0
      tcost = 0
      u_hat_old = u_hat.copy()

      #backward
      _,v_t[-1],V_t[-1] = cost_final(sim_env,x_hat[-1],300.0)
      for t in reversed(range(tN)):
        F_t = approximate_F(sim_env,x_hat[t],u_hat[t])
        #print(F_t)
        # if t == tN-1:#final
        #   l,l_x,l_xx = cost_final(sim_env,x_hat[t])
        #   c_t = np.hstack([l_x,np.zeros([adim,])])
        #   C_t = np.concatenate([l_xx,np.zeros([xdim,adim])],axis=1)
        #   C_t = np.concatenate([C_t,np.zeros([adim,xdim+adim])],axis = 0)
        # else:
        _,l_x,l_xx,l_u,l_uu,l_ux = cost_inter(sim_env,x_hat[t],u_hat[t])
        c_t = np.hstack([l_x,l_u])
        C_t = np.concatenate([l_xx,np.transpose(l_ux)],axis = 1)
        temp = np.concatenate([l_ux,l_uu],axis = 1)
        C_t = np.concatenate([C_t,temp],axis = 0)
        #print(C_t)
        Q_t = C_t+np.matmul(np.matmul(np.transpose(F_t),V_t[t+1]),F_t)#Q_t = C_t+F_t^T*V_{t+1}*F_t
        q_t = c_t+np.matmul(np.transpose(F_t),v_t[t+1])#q_t = c_t+F_t^T*v_{t+1}
        K_t[t] = -np.matmul(np.linalg.pinv(Q_t[xdim:,xdim:]),Q_t[xdim:,:xdim])
        k_t[t] = -np.matmul(np.linalg.pinv(Q_t[xdim:,xdim:]),q_t[xdim:])
        V_t[t] = Q_t[:xdim,:xdim]+np.matmul(Q_t[:xdim,xdim:],K_t[t])+np.matmul(np.transpose(K_t[t]),Q_t[xdim:,:xdim])+np.matmul(np.matmul(np.transpose(K_t[t]),Q_t[xdim:,xdim:]),K_t[t])
        v_t[t] = q_t[:xdim]+np.matmul(Q_t[:xdim,xdim:],k_t[t])+np.matmul(np.transpose(K_t[t]),q_t[xdim:])+np.matmul(np.matmul(np.transpose(K_t[t]),Q_t[xdim:,xdim:]),k_t[t])
      
    # x_0 = env.state.copy()
    # adim = env.action_space.shape[0]
    # xdim = x_0.shape[0]
    # x_hat = np.zeros([tN+1,xdim])
    # u_hat = np.zeros([tN,adim])
    # kk_seq = np.zeros([tN,adim,xdim])
    # k_seq = np.zeros([tN,adim])
    # v = np.zeros([tN+1,])
    # v_x = np.zeros([tN+1,xdim])
    # v_xx = np.zeros([tN+1,xdim,xdim])

    # for i in range(int(max_iter)):
    #   print("iteration: %d"%(i+1))
    #   rtotal = 0
    #   tcost = 0
    #   u_hat_old = u_hat.copy()

    #   #backward
    #   v[-1],v_x[-1],v_xx[-1] = cost_final(sim_env,x_hat[-1])
    #   for t in reversed(range(tN)):
    #     F_t = approximate_F(sim_env,x_hat[t],u_hat[t])
    #     f_x_t = F_t[:,:xdim]
    #     f_u_t = F_t[:,xdim:]
    #     _,l_x,l_xx,l_u,l_uu,l_ux = cost_inter(sim_env,x_hat[t],u_hat[t])
    #     q_x = l_x + np.matmul(f_x_t.T,v_x[t+1])
    #     q_u = l_u + np.matmul(f_u_t.T, v_x[t + 1])
    #     q_xx = l_xx + np.matmul(np.matmul(f_x_t.T,v_xx[t+1]),f_x_t)
    #     tmp = np.matmul(f_u_t.T, v_xx[t + 1])
    #     q_uu = l_uu + np.matmul(tmp, f_u_t)
    #     q_ux = l_ux + np.matmul(tmp, f_x_t)
    #     inv_q_uu = np.linalg.pinv(q_uu)
    #     k = -np.matmul(inv_q_uu, q_u)
    #     kk = -np.matmul(inv_q_uu, q_ux)
    #     dv = 0.5 * np.matmul(q_u, k)
    #     v[t] += dv
    #     v_x[t] = q_x - np.matmul(np.matmul(q_u, inv_q_uu), q_ux)
    #     v_xx[t] = q_xx + np.matmul(q_ux.T, kk)
    #     kk_seq[t] = kk.copy()
    #     k_seq[t] = k.copy()
      #forward
      
      sim_env.state = x_0.copy()
      x = x_0.copy()
      for t in range(tN):
        #sim_env.render()
        delta_x = x-x_hat[t]
        x_hat[t] = x.copy()
        u_hat[t] = np.matmul(K_t[t],delta_x)+k_t[t]+u_hat[t]
        x,r,done,_ = sim_env.step(u_hat[t])
        # if t != tN-1:
        cost,_,_,_,_,_=cost_inter(sim_env,x,u_hat[t])
        # else:
        #   cost,_,_ = cost_final(sim_env,x)
        rtotal+=r
        tcost+=cost
      x_hat[tN] = x.copy()
      tcost+=cost_final(sim_env,x)[0]
      print("total reward:",rtotal)
      print("total cost:",tcost)
      total_cost.append(tcost)
      total_reward.append(rtotal)
      if np.abs(u_hat-u_hat_old).max()<1e-3 :
        break
      
      # sim_env.state = x_0.copy()
      # x_seq = x_hat.copy()
      # u_seq = u_hat.copy()
      # for t in range(tN):
      #   #sim_env.render()
      #   delta_x = x_hat[t]-x_seq[t]
      #   u_hat[t] = np.matmul(kk_seq[t],delta_x)+k_seq[t]+u_seq[t]
      #   x_hat[t+1],r,done,_ = sim_env.step(u_hat[t])
      #   # if t != tN-1:
      #   cost,_,_,_,_,_=cost_inter(sim_env,x_hat[t],u_hat[t])
      #   # else:
      #   #   cost,_,_ = cost_final(sim_env,x)
      #   rtotal+=r
      #   tcost+=cost
      # tcost+=cost_final(sim_env,x_hat[tN])[0]
      # print("total reward:",rtotal)
      # print("total cost:",tcost)
      # if np.abs(u_hat-u_hat_old).max()<1e-4 :
      #   break
    plt.figure()
    plt.subplot(121)
    plt.plot(total_cost)
    plt.title("cost")
    plt.subplot(122)
    plt.plot(total_reward)
    plt.title("reward")
    return u_hat.copy()


In [13]:

env_name = "TwoLinkArm-v0"

env = TwoLinkArmEnv()
sim_env = TwoLinkArmEnv()
done = False
state = env.reset()
sim_env.reset()
treward = 0
q = []
q_dot = []
u_t = []
u_limit = []
step_num = 0
tN = 100
#env._step(np.random.uniform(-100.0,100.0,(2,)),0.001)
# env.state = np.hstack([env.goal_q,env.goal_dq])
# print (env.state)
while not done and step_num<tN:
    env.render()
    q.append(env.position)
    q_dot.append(env.velocity)
    u = calc_lqr_input(env,sim_env)
    state,reward,done,_ = env.step(u)
    treward+=reward
    step_num+=1
    u_t.append(u)
    u_limit.append(np.clip(u,env.action_space.low,env.action_space.high))
    if np.max(env.velocity)>1000:
        break
    
q = np.array(q)
q_dot = np.array(q_dot)
u_t = np.array(u_t)
u_limit = np.array(u_limit)
print("total reward:",treward)
print("total steps:",step_num)
plt.subplot(421)
plt.plot(q[:,0])
plt.title("q[0]")
plt.subplot(422)
plt.plot(q[:,1])
plt.title("q[1]")
plt.subplot(423)
plt.plot(q_dot[:,0])
plt.title("q_dot[0]")
plt.subplot(424)
plt.plot(q_dot[:,1])
plt.title("q_dot[1]")
plt.subplot(425)
plt.plot(u_t[:,0])
plt.title("u[0]")
plt.subplot(426)
plt.plot(u_t[:,1])
plt.title("u[1]")
plt.subplot(427)
plt.plot(u_limit[:,0])
plt.title("u_limit[0]")
plt.subplot(428)
plt.plot(u_limit[:,1])
plt.title("u_limit[1]")
plt.show()
env.close()
sim_env.close()


/home/mjhwang/anaconda3/envs/tf2/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


NotImplementedError: 